In [38]:
# final version
import requests
import re
import json
import numpy
import time
import datetime

def lencheck(api_res):
    if api_res[-1]['createdAt'][0:10]>=since:
        return default_len
    else:        
        return numpy.argmax(list(map(lambda x:x['createdAt'][0:10]<since,api_res)))

    
# def lencheck(api_res,datetime_str):
#     tmp = [_['createdAt'][0:10]==datetime_str for _ in api_res]
#     if True in tmp:
#         return tmp.index(True)+1
    
    
def page_load(url,end = 30):
    return requests.get(url,headers=header).json()[0:end]


class Dcard_crawler(object):
    
    def __init__(self, forum, since, until, outfile, sleep=True):
        self.forumlist = ['timecapsule','relationship','girl','makeup','dressup','funny','rainbow','marvel','boy','horoscopes','food','mood','travel']
        self.forum = forum
        self.init_url = "https://www.dcard.tw/_api/forums/{}/posts?popular=false".format(forum) if forum in self.forumlist else None
        self.since = since # if since >= '2015-01-01' else '2015-01-01'
        self.until = until if until <= str(datetime.datetime.now())[0:10] else str(datetime.datetime.now())[0:10]
        self.outfile = outfile
        self.default_len = 30
        self.sleep = sleep
        self.post_ids = list()
        self.url_num = None
        self.data = dict()
        print("Dcard_crawler is activated...")
    

    def pid_collector(self):
        assert self.init_url,"the forum you type doesn't exist"
        page = requests.get(self.init_url,headers=header).json()
        self.post_ids.append([item['id'] for item in page if self.since<=item['createdAt'][0:10]<=self.until]) 
        while True:
            iter_id = page[-1]['id']
            url = 'https://www.dcard.tw/_api/forums/{}/posts?popular=false&before={}'.format(self.forum,iter_id)
            page = page_load(url,lencheck(page))
            self.post_ids.append([item['id'] for item in page if self.since<=item['createdAt'][0:10]<=self.until])
            if lencheck(page) == default_len:
                time.sleep(2)
            else:
                break
        self.post_ids = sum(self.post_ids,[])
        self.url_num = len(self.post_ids)
        print(len(self.post_ids),"post urls are collected")
        
        
    def crawler(self):
        assert len(self.post_ids) > 0,'please run pid_collector first'
        self.data['info'] = {'forum':self.forum,'since':self.since,'until':self.until}
        self.data['post'] = [None]*self.url_num
        for idx,pid in enumerate(self.post_ids):
            # start of post crawler
            if idx % 20 == 0: print('crawling post {}...'.format(idx))
            p_url = 'https://www.dcard.tw/_api/posts/{}?'.format(pid)
            pos = requests.get(p_url,headers=header).json()
            time.sleep(2)
            # start of comment crawler
            total_pages = pos['commentCount']//30 + 1
            c_urls = ['https://www.dcard.tw/_api/posts/{}/comments?after={}'.format(pid,page*30) for page in range(total_pages)]
            c_res = [page_load(c_url) for c_url in c_urls]
            pos['comments'] = sum(c_res,[])
            self.data['post'][idx] = pos
        json.dump(self.data, open(self.outfile, 'w', encoding='utf-8'))
        print('process complete.')

In [39]:
header = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'}
forum = 'travel'
since = '2017-03-30'
until = '2017-03-31'
default_len = 30

c = Dcard_crawler(forum,since,until,'test')
c.pid_collector()
c.crawler()

Dcard_crawler is activated...
38 post urls are collected
crawling post 0...
crawling post 20...
process complete.


In [47]:
import pandas as pd
import json

pd.read_json(open('test'))

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.